In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import csv
import joblib

In [3]:
# Veri yükleme
file_path = 'C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/Augmented_Labeled_Data.csv'
data = pd.read_csv(file_path)

In [3]:
data.head()

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,AccX.1,AccY.1,AccZ.1,GyroX.1,...,GyroY.58,GyroZ.58,AccX.59,AccY.59,AccZ.59,GyroX.59,GyroY.59,GyroZ.59,Label,Activity
0,9.935937,0.833182,0.921768,-0.231822,0.010658,-0.083936,9.957485,0.545878,1.005565,-0.148420,...,-0.569963,-0.347201,10.967838,1.735796,1.154005,-0.148420,-0.174799,-0.406355,0,HIGHKNEES
1,9.959879,1.094150,0.531513,-0.125770,0.009326,-0.021850,10.005369,1.005565,0.550666,-0.132965,...,-0.285115,-0.134564,10.019734,-0.057461,1.805228,0.401825,-0.360524,-0.114046,0,HIGHKNEES
2,9.923966,1.201889,0.684742,-0.077541,0.013856,-0.014922,9.933543,1.211466,0.656011,-0.077008,...,-0.306432,-0.176132,9.425972,0.102951,2.750938,-0.151351,-0.197715,-0.225427,0,HIGHKNEES
3,9.967062,1.067814,0.459687,-0.094328,-0.012790,-0.008260,10.067618,1.039083,0.430956,-0.104986,...,-0.051960,-0.225427,9.308656,-0.404620,2.513912,0.024248,-0.356527,-0.209972,0,HIGHKNEES
4,10.067618,0.802058,0.423774,-0.087400,0.022916,-0.043700,10.022128,0.711078,0.462081,-0.042368,...,-0.062352,-0.259002,10.347740,-0.059855,1.161188,-0.237685,-0.238750,-0.322420,0,HIGHKNEES


In [11]:
# Özellikler ve etiketlerin ayrılması
X = data.drop(['Label', 'Activity'], axis=1)
y = data['Activity']
print(y)

0       HIGHKNEES
1       HIGHKNEES
2       HIGHKNEES
3       HIGHKNEES
4       HIGHKNEES
          ...    
1995        SQUAT
1996        SQUAT
1997        SQUAT
1998        SQUAT
1999        SQUAT
Name: Activity, Length: 2000, dtype: object


In [10]:
# Etiketleri sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

[0 0 0 ... 4 4 4]


In [6]:
# Her sınıf için veriyi bölme
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

In [7]:
for label in np.unique(y_encoded):
    X_label = X[y_encoded == label]
    y_label = y_encoded[y_encoded == label]
    X_train_label, X_test_label, y_train_label, y_test_label = train_test_split(
        X_label, y_label, test_size=0.3, random_state=42)
    X_train_list.append(X_train_label)
    X_test_list.append(X_test_label)
    y_train_list.append(y_train_label)
    y_test_list.append(y_test_label)

In [8]:
# Eğitim ve test setlerini birleştirme
X_train = pd.concat(X_train_list)
X_test = pd.concat(X_test_list)
y_train = np.concatenate(y_train_list)
y_test = np.concatenate(y_test_list)

In [9]:
# Veriyi ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled)
X_test_scaled = scaler.transform(X_test)

[[ 1.10545935 -0.42493604  0.20314287 ... -0.92466717 -0.5288893
  -0.54172115]
 [ 1.22299716  0.02896445  0.63932073 ... -0.37064563 -1.23430395
  -0.61078678]
 [-1.11822714 -0.89821851  0.11293823 ... -0.09171683 -0.79292895
   2.41059087]
 ...
 [ 1.29288438  0.69336295 -0.2425813  ... -0.97343876  0.0826389
  -0.49502694]
 [-1.11822714 -0.89821851  0.11293823 ...  0.51450118 -0.6802858
   2.24708316]
 [ 0.75761126  1.02646354 -0.82713158 ... -0.38557844  0.57635683
  -0.49518182]]


In [12]:
# Keras Sequential modelini oluşturma
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [13]:
# Modeli derleme
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Modeli eğitme
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6459 - loss: 0.9553 - val_accuracy: 0.0071 - val_loss: 1.8757
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9802 - loss: 0.0569 - val_accuracy: 0.9679 - val_loss: 0.1825
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9954 - loss: 0.0144 - val_accuracy: 0.9964 - val_loss: 0.0760
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.9821 - val_loss: 0.0693
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.9714 - val_loss: 0.0677
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.9750 - val_loss: 0.0615
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 9.6306e-04 - val_accuracy: 0.9786 - val_loss: 0.0539
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 6.9663e-04 - val_accuracy: 0.9786 - v

In [15]:
# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9936 - loss: 0.0222     
Test Loss: 0.015478485263884068
Test Accuracy: 0.9966832399368286


In [16]:
# Tahminleri yapma
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [15]:
# Sınıflandırma raporunu yazdırma
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

   HIGHKNEES       0.99      1.00      1.00       104
 JUMPINGJACK       1.00      0.99      1.00       128
      PUSHUP       1.00      1.00      1.00       123
       SITUP       1.00      1.00      1.00       123
       SQUAT       1.00      1.00      1.00       125

    accuracy                           1.00       603
   macro avg       1.00      1.00      1.00       603
weighted avg       1.00      1.00      1.00       603



In [16]:
# Modeli kaydetme (sadece model mimarisini)
model.save('C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/har_model.keras')
print("Model saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/har_model.keras")

Model saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/har_model.keras


In [17]:
# Ağırlıkları kaydetme
model.save_weights('C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/har_model_weights.weights.h5')
print("Model weights saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/har_model_weights.weights.h5")

Model weights saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/har_model_weights.weights.h5


In [18]:
# Label encoder kaydetme
joblib.dump(label_encoder, 'C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/label_encoder.pkl')
print("Label encoder saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/label_encoder.pkl")

Label encoder saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/label_encoder.pkl


In [19]:
# Scaler'ı kaydetme
joblib.dump(scaler, 'C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/scaler.pkl')
print("Scaler saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/scaler.pkl")

Scaler saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/scaler.pkl


In [20]:
# Test verilerini kaydetme
test_data = pd.DataFrame(X_test)
test_data['Activity'] = y_test
test_data.to_csv('C:/Users/musta/OneDrive/Masaüstü/HAR Updates/HAR_Data/test_data.csv', index=False)
print("Test data saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/test_data.csv")

Test data saved to C:/Users/Asus-Pc/Desktop/HAR Updates/HAR_Data/test_data.csv
